In [1]:
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os
import time

# Load environment variables and New York Times API key
load_dotenv()
api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("IMDB_API_KEY")

# New York Times Article API URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

query = "love"
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for movie reviews published between a begin and end date

begin_date = "20130101"
end_date = "20230531"

query_url = (
    f"{url}api-key={api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)


In [2]:
response = requests.get(query_url)
response

<Response [200]>

In [3]:
# Create an empty list to store the reviews
reviews_list = []


# Loop through 20 pages
for page_num in range(20):
    # Extend the query_url to include the page parameter
    query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&q=love&page={page_num}&api-key={api_key}"

    # Make a GET request to retrieve the page of results
    response = requests.get(query_url)
    reviews = response.json()

    # Add a 12-second interval between queries
    time.sleep(12)

    try:
        # Loop through the reviews and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
        
        # Print the query page number
        print(f"Query page {page_num} completed.")
    
    except:
        # Print the page number that had no results
        print(f"No results on page {page_num}.")

print("All reviews retrieved successfully.")


Query page 0 completed.
Query page 1 completed.
Query page 2 completed.
Query page 3 completed.
Query page 4 completed.
Query page 5 completed.
Query page 6 completed.
Query page 7 completed.
Query page 8 completed.
Query page 9 completed.
Query page 10 completed.
Query page 11 completed.
Query page 12 completed.
Query page 13 completed.
Query page 14 completed.
Query page 15 completed.
Query page 16 completed.
Query page 17 completed.
Query page 18 completed.
Query page 19 completed.
All reviews retrieved successfully.


In [4]:
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [5]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

from pandas import json_normalize

reviews_list_df = json_normalize(reviews_list)


In [6]:
# Define a function to extract the title
def extract_title(headline):
    start_index = headline.find('\u2018') + len('\u2018')  # Find the start of the title
    end_index = headline.find('\u2019 Review') + len('\u2019 Review')  # Find the end of the title
    return headline[start_index:end_index]

# Apply the function to the "headline.main" column to create the new "title" column
reviews_list_df['title'] = reviews_list_df['headline.main'].apply(extract_title)

reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’ Review
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’ Review
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’ Review
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’ Review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,


In [8]:
def extract_keywords(keyword_list):
    try:
        # Check if the input list is not empty
        if keyword_list:
            # Extract 'name' and 'value' from each dictionary item
            extracted_keywords = "; ".join([f"{item['name']}: {item['value']}" for item in keyword_list])
            return extracted_keywords
    except (KeyError, TypeError):
        pass
    return ""
# Apply the function to convert the "keywords" column from a list to a string
reviews_list_df['keywords'] = reviews_list_df['keywords'].apply(extract_keywords)

reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’ Review
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies; persons: Kapur, Shekhar; pers...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’ Review
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies; creative_works: A Tourist's G...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’ Review
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies; persons: Zlotowski, Rebecca; ...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’ Review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies; creative_works: The Other Hal...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies; creative_works: The Ottoman L...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie); persons:...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies; creative_works: Everybody Lov...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,


In [7]:
# Create a list from the "title" column
titles_list = reviews_list_df['title'].to_list()

# Display the list of titles
print(titles_list)


['The Attachment Diaries’ Review', '', 'You Can Live Forever’ Review', 'A Tourist’s Guide to Love’ Review', 'Other People’s Children’ Review', 'One True Loves’ Review', 'The Lost Weekend: A Love Story’ Review', 'A Thousand and One’ Review', 'Your Place or Mine’ Review', 'Love in the Time of Fentanyl’ Review', 'Pamela, a Love Story’ Review', 'In From the Side’ Review', 'After Love’ Review', 'Alcarràs’ Review', 'Nelly & Nadine’ Review', 'Lady Chatterley’s Lover’ Review', 'The Sound of Christmas’ Review', 'The Inspection’ Review', 'Bones and All’ Review', 'My Policeman’ Review', 'About Fate’ Review', 'Waiting for Bojangles’ Review', 'I Love My Dad’ Review', 'A Love Song’ Review', 'Alone Together’ Review', 'Art of Love’ Review', 'The Wheel’ Review', 'Thor: Love and Thunder’ Review', 'Both Sides of the Blade’ Review', 'Fire of Love’ Review', 'Love & Gelato’ Review', 'Stay Prayed Up’ Review', 'Benediction’ Review', 'Dinner in America’ Review', 'In a New York Minute’ Review', 'Anaïs in Love’ 

In [18]:
# Set up your TMDB API key

load_dotenv()
tmdb_api_key = os.getenv("IMDB_API_KEY")

# The Movie Database
url = "https://api.themoviedb.org/3/search/movie?query="

tmdb_key_string = f"&api_key={tmdb_api_key}"

response = requests.get(url + tmdb_key_string)
print(response)

<Response [200]>


In [19]:
tmdb_movies_list = []

In [20]:
# Initialize request counter
request_counter = 1

# Loop through titles list
for title in titles_list:
    # Increment request counter
    request_counter += 1
    
    # Check if request_counter is a multiple of 50
    if request_counter % 50 == 0:
        # Print message
        print("Reached a multiple of 50. Sleeping for 1 second...")
        # Sleep for 1 second
        time.sleep(1)

Reached a multiple of 50. Sleeping for 1 second...
Reached a multiple of 50. Sleeping for 1 second...
Reached a multiple of 50. Sleeping for 1 second...
Reached a multiple of 50. Sleeping for 1 second...


In [21]:
for title in titles_list:
    # Perform a GET request to The Movie Database search
    search_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    search_response = requests.get(search_url)
    search_results = search_response.json()
    
    try:
        # Collect movie ID from the first result
        movie_id = search_results['results'][0]['id']
        
        # Make a GET request using movie ID to retrieve full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_details = movie_response.json()
        
        # Extract genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]
        
        # Extract spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
        
        # Extract production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]
        
        # Create a dictionary with movie details
        movie_dict = {
            'title': movie_details['title'],
            'original_title': movie_details['original_title'],
            'budget': movie_details['budget'],
            'original_language': movie_details['original_language'],
            'homepage': movie_details['homepage'],
            'overview': movie_details['overview'],
            'popularity': movie_details['popularity'],
            'runtime': movie_details['runtime'],
            'revenue': movie_details['revenue'],
            'release_date': movie_details['release_date'],
            'vote_average': movie_details['vote_average'],
            'vote_count': movie_details['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        # Append dictionary to tmdb_movies_list
        tmdb_movies_list.append(movie_dict)
        
        # Print movie title and message
        print(f"Movie '{title}' found.")
        
    except IndexError:
        print(f"No results found for '{title}'.")
    
    # Add a 1-second interval to avoid API rate limits
    time.sleep(1)

# Print the first 5 movies in tmdb_movies_list
print(tmdb_movies_list[:5])

No results found for 'The Attachment Diaries’ Review'.
No results found for ''.
No results found for 'You Can Live Forever’ Review'.
No results found for 'A Tourist’s Guide to Love’ Review'.
No results found for 'Other People’s Children’ Review'.
No results found for 'One True Loves’ Review'.
No results found for 'The Lost Weekend: A Love Story’ Review'.
No results found for 'A Thousand and One’ Review'.
No results found for 'Your Place or Mine’ Review'.
No results found for 'Love in the Time of Fentanyl’ Review'.
No results found for 'Pamela, a Love Story’ Review'.
No results found for 'In From the Side’ Review'.
No results found for 'After Love’ Review'.
No results found for 'Alcarràs’ Review'.
Movie 'Nelly & Nadine’ Review' found.
No results found for 'Lady Chatterley’s Lover’ Review'.
No results found for 'The Sound of Christmas’ Review'.
No results found for 'The Inspection’ Review'.
No results found for 'Bones and All’ Review'.
No results found for 'My Policeman’ Review'.
No resu

In [24]:
import requests
import time

# Function to search for movie details on TMDb
def search_movie(title):
    # Construct the search URL
    search_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    
    # Make a GET request to search for the movie
    response = requests.get(search_url)
    
    # Pause for a moment to avoid hitting the rate limit
    time.sleep(1)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        movie_data = response.json()
        
        # Check if any movie was found
        if movie_data['total_results'] > 0:
            # Extract relevant movie details from the first result
            movie_details = {
                'title': movie_data['results'][0]['title'],
                'release_date': movie_data['results'][0]['release_date'],
                'overview': movie_data['results'][0]['overview'],
                'genre_ids': movie_data['results'][0]['genre_ids'],
                'original_language': movie_data['results'][0]['original_language'],
                }
            
            return movie_details
        else:
            print(f"No movie found for the title: {title}")
            return None
    else:
        print(f"Error searching for movie: {response.status_code}")
        return None

# List to store movie details
tmdb_movies_list = []

# Loop through the titles and search for each movie
for title in titles_list:
    # Search for the movie details
    movie_details = search_movie(title)
    
    # If movie details were found, append them to the list
    if movie_details:
        tmdb_movies_list.append(movie_details)

# Print the list of movie details
for movie in tmdb_movies_list:
    print(movie)


No movie found for the title: The Attachment Diaries’ Review
No movie found for the title: 
No movie found for the title: You Can Live Forever’ Review
No movie found for the title: A Tourist’s Guide to Love’ Review
No movie found for the title: Other People’s Children’ Review
No movie found for the title: One True Loves’ Review
No movie found for the title: The Lost Weekend: A Love Story’ Review
No movie found for the title: A Thousand and One’ Review
No movie found for the title: Your Place or Mine’ Review
No movie found for the title: Love in the Time of Fentanyl’ Review
No movie found for the title: Pamela, a Love Story’ Review
No movie found for the title: In From the Side’ Review
No movie found for the title: After Love’ Review
No movie found for the title: Alcarràs’ Review
No movie found for the title: Lady Chatterley’s Lover’ Review
No movie found for the title: The Sound of Christmas’ Review
No movie found for the title: The Inspection’ Review
No movie found for the title: Bone

In [25]:
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "Nelly and Monsieur Arnaud",
        "release_date": "1995-08-23",
        "overview": "Nelly leaves her lazy, unemployed husband to work for retired judge Mr Arnaud, forty years her senior, after he offers to clear her bills for her. While she types his memoirs the two develop a close friendship, but Arnaud becomes jealous when Nelly begins dating his good-looking young publisher.",
        "genre_ids": [
            18,
            10749
        ],
        "original_language": "fr"
    },
    {
        "title": "Love, Divided",
        "release_date": "2024-04-12",
        "overview": "She\u2019s a young pianist preparing for an audition. He is a games inventor who can only concentrate in complete silence. Separated by only a paper-thin wall in conjoined apartments, will they learn to live in harmony with one another? Will romance blossom through the wall?",
        "genre_ids": [
            10749,
            35
        ],
        "original_language": "es"


In [26]:
# Convert the list of dictionaries to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)

# Display the DataFrame
print(tmdb_movies_df.head())

                        title release_date  \
0   Nelly and Monsieur Arnaud   1995-08-23   
1               Love, Divided   2024-04-12   
2         Putting It Together   2001-10-14   
3       Asia: Music in Review   2006-01-01   
4  Emmanuelle: Queen of Sados   1980-02-01   

                                            overview     genre_ids  \
0  Nelly leaves her lazy, unemployed husband to w...   [18, 10749]   
1  She’s a young pianist preparing for an auditio...   [10749, 35]   
2  An all-star cast performs the music of one of ...      [18, 35]   
3  Pop-rock supergroup Asia is the focus of this ...            []   
4  A woman struggles to keep her stepdaughter fro...  [80, 18, 53]   

  original_language  
0                fr  
1                es  
2                en  
3                en  
4                el  


In [27]:
# Merge the New York Times reviews and TMDB DataFrames on the title column
merged_df = pd.merge(reviews_list_df, tmdb_movies_df, on='title', how='inner')

# Display the merged DataFrame
print(merged_df)

Empty DataFrame
Columns: [web_url, snippet, source, keywords, pub_date, word_count, headline.main, headline.kicker, headline.content_kicker, headline.print_headline, headline.name, headline.seo, headline.sub, byline.original, byline.person, byline.organization, title, release_date, overview, genre_ids, original_language]
Index: []

[0 rows x 21 columns]


In [34]:
# Define the columns that need fixing
columns_to_fix = ['genre_ids', 'original_language']

# Define the characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through columns_to_fix
for col in columns_to_fix:
    # Convert the column to string
    merged_df[col] = merged_df[col].astype(str)
    # Loop through characters_to_remove and remove them from the column
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Print the head of the updated DataFrame
print(merged_df.head())

Empty DataFrame
Columns: [web_url, snippet, source, keywords, pub_date, word_count, headline.main, headline.kicker, headline.content_kicker, headline.print_headline, headline.name, headline.seo, headline.sub, byline.original, byline.person, byline.organization, title, release_date, overview, genre_ids, original_language]
Index: []

[0 rows x 21 columns]


In [35]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)
print(merged_df.head())

Empty DataFrame
Columns: [web_url, snippet, source, keywords, pub_date, word_count, headline.main, headline.kicker, headline.content_kicker, headline.print_headline, headline.name, headline.seo, headline.sub, byline.original, byline.organization, title, release_date, overview, genre_ids, original_language]
Index: []


In [36]:
# Delete duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

In [37]:
# Export data to CSV without the index
merged_df.to_csv("merged_data.csv", index=False)